In [763]:
import pandas as pd
import os

## 해당 코드는 데이터를 전처리한 과정을 보여주는 과정입니다. 
## 블룸버그에서 가져온 데이터를 모두 정리해서 국면에 넣어줄 형태를 만들어주는 코드입니다.
## 가장 먼저 시행해야하는 코드입니다.

In [811]:
path = 'C:/Users/User/ficc_quant/data'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xlsx')] 

#### 데이터 파일 목록은 다음과 같습니다.

In [812]:
file_list_py

['Chicago FED Adjusted.xlsx',
 'corporate_bond.xlsx',
 'dollar.xlsx',
 'gold.xlsx',
 'gover_3month.xlsx',
 'highyield_spread(CSI BARC Index).xlsx',
 'high_yield_real.xlsx',
 'Investment Grade.xlsx',
 'TED Spread.xlsx',
 'TIPS.xlsx',
 'US treasury_bond.xlsx',
 'vix.xlsx']

### 데이터를 모두 붙혀줍니다.

In [813]:
file_list_py
name_list=[]
for index,file in enumerate(file_list_py):
    
    file_data=pd.read_excel(path+'/'+file)
    name=file_data.iloc[0].index[1]
    file_data=pd.DataFrame(file_data.iloc[5:,:2]).reset_index(drop=True).iloc[::-1]
    file_data.columns=['Date',name]
    
    file_data.loc[:,name]=file_data.loc[:,name]
    file_data=file_data.reset_index(drop=True)
    #break
    #print(file_data.iloc[0].index[1])
    name_list.append(name)
    print(name,index)
    if index>0:
        all_file_data=pd.merge(all_file_data,file_data, how='outer')
    else:
        all_file_data=file_data
    #print(file_data)
    
    
    

NFCIADJ  Index                                                   0
LUACTRUU Index                                                   1
DXY Curncy                                                       2
XAU BGN Curncy                                                   3
USGG3M   Index                                                   4
CSI BARC Index                                                   5
I14031US Index                                                   6
LUACOAS  Index                                                   7
US0003M  Index                                                   8
TIPS 9
LUATTRUU Index                                                   10
VIX Index                                                        11


In [814]:
all_file_data.Date=pd.to_datetime(all_file_data.Date,errors='coerce')#.to_excel('버그.xlsx')
all_file_data=all_file_data.loc[all_file_data.Date != None].reset_index(drop=True)#.iloc[:-2,:]
all_file_data=all_file_data.sort_values(by=['Date'])#.iloc[:-1,:]#.to_excel('all_data.xlsx')
all_file_data=all_file_data.ffill()
index_data=all_file_data#.dropna()

### 데이터를 모두 붙혀주면 다음과 같이null 값이 많습니다. 이를 전처리해줍니다.

In [815]:
index_data

,Date,NFCIADJ Index,LUACTRUU Index,DXY Curncy,XAU BGN Curncy,USGG3M Index,CSI BARC Index,I14031US Index,LUACOAS Index,US0003M Index,TIPS,LUATTRUU Index,VIX Index
8612,1973-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.99,NaN
8613,1973-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.73,NaN
8614,1973-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.73,NaN
8615,1973-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.66,NaN
8616,1973-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.88,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6303,2023-01-10,-0.25595,3028.97,103.236,1877.03,4.5845,4.73,399.2509,1.28,4.80586,329.7089,2220.21,20.58
6304,2023-01-11,-0.25595,3052.29,103.188,1875.69,4.6565,4.71,401.0719,1.26,4.81500,329.7089,2230.83,21.09
6305,2023-01-12,-0.25595,3079.12,102.246,1897.09,4.5755,4.65,402.3142,1.25,4.82971,329.7089,2246.33,18.83
6306,2023-01-13,-0.25595,3069.92,102.204,1920.23,4.5732,4.65,404.4192,1.24,4.82971,329.7089,2238.14,18.35


In [816]:
index_data=index_data.dropna().iloc[:-5].drop_duplicates('Date').set_index('Date').asfreq('w-fri').loc['2000-12':]

### Ted spread를 만들어줍니다.

In [817]:
index_data.columns=['condition'	,'corporate','dollar','gold','short_corporate','spread','highyield','investment_grade','Libor','TIPS','US_bond','VIX Index']                                                       
index_data.condition=index_data.condition+1
index_data.loc[:,'Ted_spread']=index_data.loc[:,'short_corporate']-index_data.Libor


### 미리 인덱스 값을 저장해줍니다.

In [818]:
date_col=index_data.index

### 가격의변동률을 저장해줍니다. 이후 백테스팅이나 비중 최적화에 쓰일 데이터입니다.

In [819]:
index_data.pct_change().iloc[1:].loc[:,['condition'	,'corporate','dollar','gold','short_corporate','spread','highyield','investment_grade','Libor','TIPS','US_bond','VIX Index','Ted_spread']   ].to_excel('price_change.xlsx')


### 다음은 국면을 판별할때 필요한 데이터를 모아놓은 데이터를 저장해줍니다.(all_index_data.xlsx)
### 차분을 해줘야합니다.

In [820]:
index_data.columns=['condition'	,'corporate','dollar','gold','highyield','spread','investment_grade','Libor','TIPS','US_bond','VIX Index','short_corporate','Ted_spread']
index_data.index=date_col


In [822]:
index_data=index_data.fillna(1).diff().iloc[1:].ffill()
index_data=index_data.fillna(0)

In [823]:
index_data.to_excel('all_index_data.xlsx')